In diesem Notebook wird eine Delayed Streaming Klassifizierung für t_agr_a mit V = 7000 durchgeführt

In [1]:
import numpy as np
import pandas as pd

from skmultiflow.trees import HoeffdingTreeClassifier
from skmultiflow.trees import HoeffdingAdaptiveTreeClassifier
from skmultiflow.trees import ExtremelyFastDecisionTreeClassifier
from skmultiflow.meta import AdaptiveRandomForestClassifier
from skmultiflow.bayes import NaiveBayes
from skmultiflow.meta import AccuracyWeightedEnsembleClassifier
from skmultiflow.lazy import KNNADWINClassifier

from skmultiflow.data import TemporalDataStream

from skmultiflow.evaluation import EvaluatePrequentialDelayed

In [2]:
import warnings
warnings.filterwarnings('ignore')

#set columns
attribute = ['salary', 'comission', 'age', 'elevel',
                 'car', 'zipcode', 'hvalue', 'hyears', 'loan']
target = 'class'
timestamp = 'datetime'

#read csv 
t_agr_a = pd.read_csv("data/t_agr_a.csv", )
t_agr_a = t_agr_a.loc[:, ~t_agr_a.columns.str.contains('^Unnamed')]

#convert time to datetime
t_agr_a[timestamp] = pd.to_datetime(t_agr_a[timestamp])

#get X, y and time
X = t_agr_a[attribute].values
y = t_agr_a[target].values
time = t_agr_a[timestamp].values

#set delay
delay_time = 7000

#create stream
t_agr_a_stream = TemporalDataStream(X, y, time, sample_delay=delay_time, ordered=False)
nominal=['elevel', 'car', 'zipcode']

#set classifier
ht = HoeffdingTreeClassifier(leaf_prediction='nb', nominal_attributes=nominal)
hat = HoeffdingAdaptiveTreeClassifier(nominal_attributes=nominal)
efdt = ExtremelyFastDecisionTreeClassifier(nominal_attributes=nominal)
arf = AdaptiveRandomForestClassifier(nominal_attributes=nominal)
snb = NaiveBayes(nominal_attributes=nominal)
awe = AccuracyWeightedEnsembleClassifier(n_estimators=15, base_estimator=NaiveBayes(nominal_attributes=nominal))
knn_adwin = KNNADWINClassifier(n_neighbors=10)

#set evaluator
evaluator = EvaluatePrequentialDelayed(
                                n_wait=10,
                                pretrain_size=100,
                                max_samples=X.shape[0],
                                output_file='results_t_agr_a.csv',
                                metrics=['accuracy', 'kappa', 
                                         'precision', 'recall', 
                                         'f1', 'running_time', 
                                         'model_size'])

#evaluate
evaluator.evaluate(stream=t_agr_a_stream, 
                   model=[ht, hat, efdt, arf, 
                          snb, awe, knn_adwin], 
                   model_names=['Hoeffding Tree', 'Hoeffding Adaptive Tree', 
                                'Extremely Fast Decision Tree', 
                                'Adaptive Random Forest', 
                                'Naive Bayes', 'Accuracy Weighted Ensembler', 'KNN ADWIN'])

Prequential Evaluation Delayed
Evaluating 1 target(s).
Pre-training on 100 sample(s).
Evaluating...
 ###################- [95%] [323.38s]Processed samples: 10000
Mean performance:
Hoeffding Tree - Accuracy     : 0.5303
Hoeffding Tree - Kappa        : -0.0066
Hoeffding Tree - Precision: 0.4295
Hoeffding Tree - Recall: 0.2366
Hoeffding Tree - F1 score: 0.3051
Hoeffding Tree - Training time (s)  : 0.94
Hoeffding Tree - Testing time  (s)  : 1.27
Hoeffding Tree - Total time    (s)  : 2.21
Hoeffding Tree - Size (kB)          : 63.1201
Hoeffding Adaptive Tree - Accuracy     : 0.5396
Hoeffding Adaptive Tree - Kappa        : 0.0110
Hoeffding Adaptive Tree - Precision: 0.4469
Hoeffding Adaptive Tree - Recall: 0.2371
Hoeffding Adaptive Tree - F1 score: 0.3098
Hoeffding Adaptive Tree - Training time (s)  : 3.30
Hoeffding Adaptive Tree - Testing time  (s)  : 1.10
Hoeffding Adaptive Tree - Total time    (s)  : 4.40
Hoeffding Adaptive Tree - Size (kB)          : 98.5283
Extremely Fast Decision Tree -

[HoeffdingTreeClassifier(binary_split=False, grace_period=200,
                         leaf_prediction='nb', max_byte_size=33554432,
                         memory_estimate_period=1000000, nb_threshold=0,
                         no_preprune=False,
                         nominal_attributes=['elevel', 'car', 'zipcode'],
                         remove_poor_atts=False, split_confidence=1e-07,
                         split_criterion='info_gain', stop_mem_management=False,
                         tie_threshold=0.05),
 HoeffdingAdaptiveTreeClassifier(binary_split=False, bootstrap_sampling=True,
                                 grace_period=200, leaf_prediction='nba',
                                 max_byte_size=33554432,
                                 memory_estimate_period=1000000, nb_threshold=0,
                                 no_preprune=False,
                                 nominal_attributes=['elevel', 'car', 'zipcode'],
                                 random_state=None,